In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

from torchvision.datasets import CIFAR10

import torchvision.models
from torch import nn, optim
import time
import copy
import numpy as np
from tqdm import tqdm

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [11]:
total_epochs = 100

batch_size = 128

# intial learning rate
Lr = 0.1

SAVE_PATH = './cifar10_save/'
filename = '{}best_cnn_model'.format(SAVE_PATH)  
 
 
torch.backends.cudnn.benchmark = True

In [4]:
# prepare data
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor()
        , transforms.RandomCrop(32, padding=4)  # First fill the surroundings with 0, then randomly crop the image to 32*32
        , transforms.RandomHorizontalFlip(p=0.5)  # Random horizontal flip Choose a probability probability
        , transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
    ]),
    'test': transforms.Compose([
        transforms.ToTensor()
        , transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

DATASET_PATH='./data_mine'

# prepare train and test
image_datasets = CIFAR10(DATASET_PATH, train=True,transform=data_transforms['train'], download=True)
image_test = CIFAR10(DATASET_PATH, train=False,transform=data_transforms['test'], download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
from torch.utils.data import random_split

#Separate 10,000 samples from the training set into the validation set
random_seed=42
torch.manual_seed(random_seed)
val_size=10000
train_size=len(image_datasets)-val_size

train_ds,val_ds=random_split(image_datasets,[train_size,val_size])

In [6]:
train_dataloader = torch.utils.data.DataLoader(train_ds,batch_size= batch_size,shuffle= True,num_workers=0)
test_dataloader = torch.utils.data.DataLoader(image_test,batch_size= batch_size,shuffle= False,num_workers=0)
val_dataloader = torch.utils.data.DataLoader(val_ds,batch_size= batch_size,shuffle= False,num_workers=0)

In [7]:
# define model
model_ft = torchvision.models.resnet18(pretrained=False)
 # modify model
model_ft.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1, bias=False)  # The first layer is changed to a 3x3 convolution kernel
model_ft.maxpool = nn.MaxPool2d(1, 1, 0)  # The image is too small and has no features, so the pooling layer is disabled by using a 1x1 pooling kernel.
num_ftrs = model_ft.fc.in_features  
model_ft.fc = nn.Linear(num_ftrs, 10)
 
model_ft.to(device)

/userhome/cs2/u3637112/anaconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/userhome/cs2/u3637112/anaconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
# create loss function
loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)


# Used to record the number of batches where the loss has not changed
counter = 0
    # record train step train\valid\test
total_step = {
    'train': 0, 'valid': 0, 'test':0
    }
    # start time
since = time.time()
    # current min loss
valid_loss_min = np.Inf
save_num = 0
best_acc = 0

In [ ]:
filename_test = 'result.txt'.format('./cifar10_save/')
with open(filename_test, 'w') as f:
    for epoch in range(total_epochs):
        # Dynamic Learning Rate
        if counter / 10 == 1:
            counter = 0
            Lr = Lr * 0.5
            
        optimizer = optim.SGD(model_ft.parameters(), lr=Lr, momentum=0.9, weight_decay=5e-4)
 
        print('Epoch {}/{}'.format(epoch + 1, total_epochs))
        print('-' * 10)
        print()
        # fist train then valid
        for phase in ['train', 'valid']:
       
            if phase == 'train':
                model_ft.train()  # train
            else:
                model_ft.eval()  # valid
            
            running_loss = 0.0
            running_corrects = 0
            
            # load data
            for inputs, labels in tqdm(train_dataloader if phase=='train' else val_dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)
 
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model_ft(inputs)
                    loss = loss_fn(outputs, labels)
                    _, preds = torch.max(outputs, 1)  
 
                    # Update weights during training
                    if phase == 'train':
                        loss.backward()  
                        optimizer.step() 
                        
 
                # loss value
                running_loss += loss.item() * inputs.size(0)  
                running_corrects += (preds == labels).sum() 
                total_step[phase] += 1
            
            # Calculate the loss rate and accuracy after one round of training
            if phase=='train':
                epoch_loss = running_loss / len(train_dataloader.sampler)  
                epoch_acc_train = float(running_corrects) / len(train_dataloader.sampler) 
            else:
                epoch_loss = running_loss / len(val_dataloader.sampler) 
                epoch_acc_val = float(running_corrects) / len(val_dataloader.sampler)  
        
            time_elapsed = time.time() - since
            print()
            print('total time {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
            print('{} Loss: {:.4f}[{}] Acc: {:.4f}'.format(phase, epoch_loss, counter, epoch_acc_train if phase=='train' else epoch_acc_val))
 
            if phase == 'valid':
                if epoch_loss < valid_loss_min:  # epoch_acc > best_acc:
                    best_acc = epoch_acc_val
 
                   
                    best_model_wts = copy.deepcopy(model_ft.state_dict())
                    state = {
                        'state_dict': model_ft.state_dict(),
                        'best_acc': best_acc,
                        'optimizer': optimizer.state_dict(),
                    }
                    # Only save the last two training results
                    save_num = 0 if save_num > 1 else save_num
                    save_name_t = '{}_{}.pth'.format(filename, save_num)
                    torch.save(state, save_name_t)  
                    print("best model has been saved, accuracy: \033[1;31m {:.2f}%\033[0m，filename:{}".format(best_acc * 100, save_name_t))
                    save_num += 1
                    valid_loss_min = epoch_loss
                    counter = 0
                else:
                    counter += 1
 
        print()
        print('current learning rate: {:.7f}'.format(optimizer.param_groups[0]['lr']))
        print()    

        running_loss = 0.0
        running_corrects = 0
        
        model_ft.eval()
        for inputs, labels in tqdm(test_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model_ft(inputs)
                loss = loss_fn(outputs, labels)
                _, preds = torch.max(outputs, 1) 
 
            running_loss += loss.item() * inputs.size(0) 
            running_corrects += (preds == labels).sum()  
            total_step['test'] += 1
       
        epoch_loss = running_loss / len(test_dataloader.sampler)  
        epoch_acc_test = float(running_corrects) / len(test_dataloader.sampler)  
       
        time_elapsed = time.time() - since
        print()
        print('total time {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('{} Loss: {:.4f}[{}] Acc: {:.4f}'.format('test', epoch_loss, counter, epoch_acc_test))
        f.write(str(epoch)+" ")
        f.write(str(epoch_acc_train)+" ")
        f.write(str(epoch_acc_val)+" ")
        f.writelines(str(epoch_acc_test)+"\n")
 
    # result
    time_elapsed = time.time() - since
    print()
    print('finish!')
    print('task total time: {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('the best accuracy of testloader: {:4f}'.format(best_acc))
    save_num = save_num - 1
    save_num = save_num if save_num < 0 else 1
    save_name_t = '{}_{}.pth'.format(filename, save_num)
    print('the best model is saved in: {}'.format(save_name_t))
f.close()

Epoch 1/100
----------



100%|█████████████████████████████████████████████████████████████████████████████████| 313/313 [00:27<00:00, 11.24it/s]



total time 40m 0s
train Loss: 0.3659[1] Acc: 0.8742


100%|███████████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 16.70it/s]



total time 40m 5s
valid Loss: 0.5775[1] Acc: 0.8116

current learning rate: 0.1000000



...

100%|███████████████████████████████████████████████████████████████████████████████████| 79/79 [00:03<00:00, 23.56it/s]



total time 98m 15s
test Loss: 0.2112[3] Acc: 0.9443
Epoch 99/100
----------



100%|█████████████████████████████████████████████████████████████████████████████████| 313/313 [00:30<00:00, 10.31it/s]



total time 98m 45s
train Loss: 0.0020[3] Acc: 1.0000


 71%|██████████████████████████████████████████████████████████▊                        | 56/79 [00:03<00:01, 14.89it/s]